# Paper: Physics-Informed Graph Neural Networks for Water Distribution Systems.

## Workbook for reproducing the results in Tables 1, 2, 3 and 4.

We have included the trained models for all 5 WDS that are placed in the directory 'trained_models'.

### Initializing.

In [ ]:
%load_ext autoreload
%autoreload 2
from utils.utils import *
from dataset_generator import run
import pandas as pd
import json, argparse, warnings, os
import torch
warnings.filterwarnings('ignore')

""" Specifying the directory for trained models. """
file_dir = os.getcwd()
save_dir = os.path.join(file_dir, "trained_models")

""" Creating an output file to log progress. """
out_f = open(save_dir+"/output_"+str(datetime.date.today())+".txt", "a")

""" 
        Specifying parameters. 
        You can specify here, which WDS you want to evaluate 
        from ['hanoi', 'fossolo', 'pescara', 'l_town', 'zhijiang'].
        You can also change the batch size we used for evaluation (2048)
        to a smaller values if you are evaluating on a smaller GPU. 
"""
args_dict = {
        "wdn": "l_town",
        "start_scenario": 21,
        "end_scenario": 50,
        "bias": False,
        "n_days": 14,
        "batch_size": 2048,
        "M_n": 2,
        "out_dim": 1,
        "M_e": 2,
        "I": 5,
        "n_iter": 10,
        "r_iter": 10,
        "n_mlp": 2,
        "M_l": 128
        }
args = argparse.Namespace(**args_dict)



### Generating Scenarios - Only run if not already generated and saved.

In [ ]:
# scenario_times = []

# scenario_times = run(args.wdn, args.start_scenario, args.end_scenario, scenario_times)

# wntr_sim_times = {}
# for idx, s in enumerate(range(args.start_scenario, args.end_scenario + 1)):
#     wntr_sim_times[str(s)] = scenario_times[idx]    

# wntr_sim_times_df = pd.DataFrame(wntr_sim_times, index=[0])
# wntr_sim_times_df.to_csv(save_dir + "/wntr_sim_times_" + args.wdn + '.csv')
# print(sum(scenario_times)/60)
# display(wntr_sim_times_df)

### Loading generated scenarios times.

In [ ]:

wntr_sim_times_df = pd.read_csv(save_dir + "/wntr_sim_times_" + args.wdn + '.csv')

wntr_sim_time_total = np.round(wntr_sim_times_df.sum(axis=1).values[0], 2)

print("Time taken to generate scenarios for " + args.wdn + ": ", wntr_sim_time_total, "seconds")

display(wntr_sim_times_df)

### Reading Scenarios.

In [ ]:


X_tvt, edge_indices_tvt, edge_attr_tvt, wntr_demands = [], [], [], []

""" Reading scenarios one by one. """
for s in range(args.start_scenario, args.end_scenario + 1):

    """ Specifying paths for scenario, .inp file and .xlsx file. """
    scenario_path = os.path.join(os.getcwd(),"networks",  args.wdn, "s"+str(s))
    """ Since Hanoi has all configuration (*.inp) files available (Vrachimis et al. 2018), we read directly from those.
        For the other networks, we read from the .inp files that we created while generating scenarios. These files
        have a consistent naming scheme, so we can read those using the WDS name. """
    if args.wdn == "hanoi":
        args.inp_file = os.path.join(scenario_path, "Hanoi_CMH_Scenario-" + str(s) + ".inp")
    else:
        args.inp_file = os.path.join(scenario_path, args.wdn + ".inp")
    args.path_to_data = os.path.join(scenario_path, "Results-Clean", "Measurements_All.xlsx")

    """ 
    Loading the dataset from the generated scenario. This returns a "wdn_graph" object with following attributes:
    X               A (S x N_n x 2) tensor having EPANET/WNTR estimated Heads (h_wntr) and the Original Demands (d_star).
    node_coords     Coordinates of nodes that are useful for plotting,
    node_indices    Indices of the nodes.     
    edge_indices    A (S x 2 x N_e) tensor specifying bidirectional edge connections.  
    edge_attr       A (S x N_e x 2) tensor having r and the flows estimated by EPANET/WNTR (q_wntr)
    Please note that we load h_wntr and q_wntr primarily for error computations after evaluation and also to get the
    reservoir heads (h_star). These are never used in the training of our model. 
    """
    wdn_graph, reservoirs = create_graph(args.inp_file, args.path_to_data)

    """ Saving only the number of samples specified. """
    X_s = wdn_graph.X.clone()
    edge_indices_s = wdn_graph.edge_indices.clone()
    edge_attr_s = wdn_graph.edge_attr.clone()

    """ Appending these to dataset lists. """
    X_tvt.append(X_s)
    edge_indices_tvt += list(edge_indices_s)
    edge_attr_tvt += list(edge_attr_s)

    wntr_d_df = pd.read_csv(os.path.join(scenario_path, "Results-Clean", "Measurements_All_Demands.csv"))
    wntr_d_df['Timestamp'] = pd.to_datetime(wntr_d_df['Timestamp'])#, unit='s')
    wntr_d_df = wntr_d_df.set_index("Timestamp")
    wntr_d = torch.tensor(wntr_d_df.astype("float32").values)
    wntr_demands.append(wntr_d)


    print('\nRead Scenario ', str(s))


X_tvt = torch.vstack(X_tvt)    
d_wntr = torch.vstack(wntr_demands).unsqueeze(2)    

wds_tvt = WDN_Graph(X=X_tvt, edge_indices=edge_indices_tvt, edge_attr=edge_attr_tvt)

n_nodes = wds_tvt.X.shape[1]
n_edges = wds_tvt.edge_indices[0].shape[1]
n_samples = wds_tvt.X.shape[0]

wds_tvt.X.shape, wds_tvt.edge_indices[0].shape, wds_tvt.edge_attr[0].shape, d_wntr.shape

### Computing WDS Attributes - Table 4.

In [ ]:


attrib_dict = {}

attrib_dict["No. of junctions"] = n_nodes
attrib_dict["No. of links"] = n_edges
attrib_dict["No. of reservoirs"] = len(reservoirs)

G = nx.DiGraph()
edge_list = [ (u, v) for u, v in zip(*np.array(wds_tvt.edge_indices[0])) ]
G.add_edges_from(edge_list)

attrib_dict["Diameter"] = nx.diameter(G)
node_degrees = [val for (node, val) in G.degree()]
attrib_dict["Degree_min"] = np.round(np.min(node_degrees), 2)
attrib_dict["Degree_mean"] = np.round(np.mean(node_degrees), 2)
attrib_dict["Degree_max"] = np.round(np.max(node_degrees), 2)

attrib_df = pd.DataFrame(attrib_dict, index=[args.wdn])
# attrib_df.to_csv(save_dir+'/attributes_'+ args.wdn + '.csv')
display(attrib_df)


### Inference

In [ ]:
from train_test import *
from models.models import *
import pickle, os, time

""" Initializing the model and printing the number of parameters. """
model = PI_GCN( M_n = 2,                    # number of node features (d_star, d_hat).
                out_dim = 1,                # out dimension is 1 since only flows are directly estimated.
                M_e = 2,                    # number of edge features (q_hat, q_tilde).
                M_l = args.M_l,             # specified latent dimension.
                I = args.I,                 # number of GCN layers.
                num_layers = args.n_mlp,    # number of NN layers used in every MLP.
                n_iter = args.n_iter,       # minimum number of iterations.
                bias = False                # we do not use any bias.
                ).to(device)

args.model_path = "trained_models/" + args.wdn + "/model_PI_GCN_" + args.wdn + ".pt"
model_state = torch.load(args.model_path)
model.load_state_dict(model_state["model"])

zeta = 0

start = time.time()
hd_wntr, h_tilde, q_hat, d_hat, test_losses = test(wds_tvt, model, reservoirs, args, save_dir, out_f, zeta=0)
eval_time = np.round(time.time() - start, 2)
print('Time taken for ', n_samples, ' samples: ', eval_time, ' seconds')

e_attr = torch.stack((wds_tvt.edge_attr))
e_index = torch.stack((wds_tvt.edge_indices))

d_star = wds_tvt.X[:, :, 1:2].clone()
d_hat[:, reservoirs, 0] = 0

q_wntr = e_attr[:, :, 1:2].clone()
h_wntr = hd_wntr[:, :, 0:1].clone()

d_star.shape, d_hat.shape, q_wntr.shape, q_hat.shape, h_wntr.shape, h_tilde.shape

### Creating the Results dictionary.

In [ ]:
results_dict = {}
not_outlier_parcent = .95

results_dict['No. of samples'] = n_samples
results_dict['WNTR - Generation time'] = wntr_sim_time_total
results_dict['Model - Evaluation time'] = eval_time

results_dict

### Errors Computations -  EPANET

In [ ]:
from utils.utils import mean_rel_abs_error, get_not_oulier_errors

mrae_demands_mean_samples = mean_rel_abs_error(d_star, d_wntr)
results_dict['WNTR - Mean MRAE - Demands %'] = np.round(mrae_demands_mean_samples.mean().item() * 100, 8)
results_dict['WNTR - Std MRAE - Demands %'] = np.round(mrae_demands_mean_samples.std().item() * 100, 8)
mrae_demands_mean_samples_no = get_not_oulier_errors(mrae_demands_mean_samples, not_outlier_parcent)
results_dict['WNTR - NO-Mean MRAE - Demands %'] = np.round(mrae_demands_mean_samples_no.mean().item() * 100, 8)
results_dict['WNTR - NO-Std MRAE - Demands %'] = np.round(mrae_demands_mean_samples_no.std().item() * 100, 8)

results_dict

### Errors Computations -  Model Predictions

In [ ]:


mrae_demands_mean_samples = mean_rel_abs_error(d_star, d_hat)
results_dict['Model - Mean MRAE - Demands %'] = np.round(mrae_demands_mean_samples.mean().item() * 100, 8)
results_dict['Model - Std MRAE - Demands %'] = np.round(mrae_demands_mean_samples.std().item() * 100, 8)
mrae_demands_mean_samples_no = get_not_oulier_errors(mrae_demands_mean_samples, not_outlier_parcent)
results_dict['Model - NO-Mean MRAE - Demands %'] = np.round(mrae_demands_mean_samples_no.mean().item() * 100, 8)
results_dict['Model - NO-Std MRAE - Demands %'] = np.round(mrae_demands_mean_samples_no.std().item() * 100, 8)

mrae_flows_mean_samples = mean_rel_abs_error(q_wntr, q_hat)
results_dict['Model - Mean MRAE - Flows %'] = np.round(mrae_flows_mean_samples.mean().item() * 100, 8)
results_dict['Model - Std MRAE - Flows %'] = np.round(mrae_flows_mean_samples.std().item() * 100, 8)
mrae_flows_mean_samples_no = get_not_oulier_errors(mrae_flows_mean_samples, not_outlier_parcent)
results_dict['Model - NO-Mean MRAE - Flows %'] = np.round(mrae_flows_mean_samples_no.mean().item() * 100, 8)
results_dict['Model - NO-Std MRAE - Flows %'] = np.round(mrae_flows_mean_samples_no.std().item() * 100, 8)

mrae_heads_mean_samples = mean_rel_abs_error(h_wntr, h_tilde)
results_dict['Model - Mean MRAE - Heads %'] = np.round(mrae_heads_mean_samples.mean().item() * 100, 8)
results_dict['Model - Std MRAE - Heads %'] = np.round(mrae_heads_mean_samples.std().item() * 100, 8)
mrae_heads_mean_samples_no = get_not_oulier_errors(mrae_heads_mean_samples, not_outlier_parcent)
results_dict['Model - NO-Mean MRAE - Heads %'] = np.round(mrae_heads_mean_samples_no.mean().item() * 100, 8)
results_dict['Model - NO-Std MRAE - Heads %'] = np.round(mrae_heads_mean_samples_no.std().item() * 100, 8)

results_df = pd.DataFrame(results_dict, index=[args.wdn]).transpose()
# results_df.to_csv(save_dir+'/results_'+ args.wdn + '.csv')

display(results_df)
results_dict